<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Cosine-sparse-version" data-toc-modified-id="Cosine-sparse-version-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Cosine sparse version</a></span><ul class="toc-item"><li><span><a href="#Benchmarking" data-toc-modified-id="Benchmarking-0.1.1"><span class="toc-item-num">0.1.1&nbsp;&nbsp;</span>Benchmarking</a></span></li></ul></li><li><span><a href="#Euclideans-sparse-version" data-toc-modified-id="Euclideans-sparse-version-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Euclideans sparse version</a></span><ul class="toc-item"><li><span><a href="#Benchmarking" data-toc-modified-id="Benchmarking-0.2.1"><span class="toc-item-num">0.2.1&nbsp;&nbsp;</span>Benchmarking</a></span></li></ul></li></ul></li><li><span><a href="#Batching-distance-computation" data-toc-modified-id="Batching-distance-computation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Batching distance computation</a></span></li></ul></div>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from jina import Document, DocumentArray
#from jina.math import 
import scipy.sparse as sp

In [3]:
import jina

from jina.math.distance import sparse_cosine, sparse_sqeuclidean

In [4]:
aux1 = sp.csr_matrix([0,0,0,1,0])
aux2 = sp.csr_matrix([1,1,0,1,0])

darr1 = DocumentArray([Document(embedding=aux1),Document(embedding=aux1)])
darr2 = DocumentArray([Document(embedding=aux2),Document(embedding=aux2)])

print(darr1[0].embedding.shape)
print(darr2[0].embedding.shape)

(1, 5)
(1, 5)


In [5]:
X = sp.vstack(darr1.get_attributes('embedding'))
X

<2x5 sparse matrix of type '<class 'numpy.longlong'>'
	with 2 stored elements in COOrdinate format>

Scipy cdist does not work with sparse vectors

In [6]:
import scipy
from scipy.spatial.distance import cdist
aux1 = sp.csr_matrix([0,0,0,1,0])
aux2 = sp.csr_matrix([1,1,0,1,0])
#cdist(aux1, aux2)

## Cosine sparse version

In [7]:
def sp_cosine(x_mat: 'np.ndarray', y_mat: 'np.ndarray') -> 'np.ndarray':
    """Cosine distance between each row in x_mat and each row in y_mat.
    :param x_mat: np.ndarray with ndim=2
    :param y_mat: np.ndarray with ndim=2
    :return: np.ndarray  with ndim=2
    """
    from scipy.sparse.linalg import norm
    return np.asarray(1 - x_mat.dot(y_mat.T) / np.outer(norm(x_mat, axis=1), norm(y_mat, axis=1)))

In [8]:
x_mat = sp.csr_matrix([[0,0,0,1,0],[0,0,0,1,0],[1,1,0,1,0]])
y_mat = sp.csr_matrix([[1,1,0,1,0],[0,0,0,1,0]])
sp_cosine(x_mat,y_mat)

array([[ 4.22649731e-01,  0.00000000e+00],
       [ 4.22649731e-01,  0.00000000e+00],
       [-2.22044605e-16,  4.22649731e-01]])

In [9]:
x_mat = np.array([[0,0,0,1,0],[0,0,0,1,0],[1,1,0,1,0]])
y_mat = np.array([[1,1,0,1,0],[0,0,0,1,0]])
jina.math.distance.cosine(x_mat,y_mat)

array([[ 4.22649731e-01,  0.00000000e+00],
       [ 4.22649731e-01,  0.00000000e+00],
       [-2.22044605e-16,  4.22649731e-01]])

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
x_mat = sp.csr_matrix([[0,0,0,1,0],[0,0,0,1,0],[1,1,0,1,0]])
y_mat = sp.csr_matrix([[1,1,0,1,0],[0,0,0,1,0]])
1-cosine_similarity(x_mat,y_mat)

array([[ 4.22649731e-01,  0.00000000e+00],
       [ 4.22649731e-01,  0.00000000e+00],
       [-2.22044605e-16,  4.22649731e-01]])

### Benchmarking

In [11]:
from sklearn.metrics.pairwise import cosine_distances

In [12]:
n_examples = 10_000
n_features = 10_000
x_mat = sp.rand(n_examples, n_features)
n_examples = 1_000
y_mat = sp.rand(n_examples, n_features)

In [13]:
%timeit cosine_distances(x_mat,y_mat)

226 ms ± 5.77 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%timeit sp_cosine(x_mat,y_mat)

283 ms ± 2.51 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Euclideans sparse version

In [15]:
def sp_sqeuclidean(x_mat: 'np.ndarray', y_mat: 'np.ndarray') -> 'np.ndarray':
    """Cosine distance between each row in x_mat and each row in y_mat.
    :param x_mat: np.ndarray with ndim=2
    :param y_mat: np.ndarray with ndim=2
    :return: np.ndarray  with ndim=2
    """
    return np.asarray(
        y_mat.power(2).sum(axis=1).flatten()
        + x_mat.power(2).sum(axis=1)
        - 2 * x_mat.dot(y_mat.T)
    )


In [16]:
x_mat = sp.csr_matrix([[0,0,0,2,0],[0,0,0,1,0],[1,1,0,1,0]])
y_mat = sp.csr_matrix([[2,1,0,1,0],[0,0,0,1,0]])

In [17]:
x_mat_dense = np.array(x_mat.todense())
y_mat_dense = np.array(y_mat.todense())
jina.math.distance.sqeuclidean(x_mat_dense,y_mat_dense)

array([[6, 1],
       [5, 0],
       [1, 2]], dtype=int64)

In [18]:
sp_sqeuclidean(x_mat, y_mat)

array([[6, 1],
       [5, 0],
       [1, 2]], dtype=int64)

### Benchmarking

In [19]:
from sklearn.metrics.pairwise import euclidean_distances

In [20]:
n_examples = 10_000
n_features = 10_000
x_mat = sp.rand(n_examples, n_features, density=0.05)
n_examples = 1_000
y_mat = sp.rand(n_examples, n_features)

In [21]:
%timeit sp_sqeuclidean(x_mat,y_mat)

483 ms ± 4.81 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%timeit euclidean_distances(x_mat,y_mat)

466 ms ± 7.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
np.testing.assert_almost_equal(euclidean_distances(x_mat, y_mat)**2,
                               sp_sqeuclidean(x_mat, y_mat))

# Batching distance computation


In [30]:
from jina.math import distance

In [38]:
np.random.seed(123)
X = np.random.random((10,128))
Y = np.random.random((300,128))

In [40]:
jina.math.distance.cosine(X,Y)

array([[0.23053287, 0.19060594, 0.20205071, ..., 0.2415822 , 0.21349608,
        0.1866112 ],
       [0.24950841, 0.2317442 , 0.23412835, ..., 0.21793382, 0.24647694,
        0.27346042],
       [0.23015447, 0.26612968, 0.25757846, ..., 0.23001153, 0.23994445,
        0.23484005],
       ...,
       [0.22507869, 0.20794313, 0.28994818, ..., 0.26923648, 0.25165393,
        0.3054304 ],
       [0.2503967 , 0.26948342, 0.27601632, ..., 0.2680446 , 0.32414386,
        0.23514155],
       [0.26868149, 0.27516195, 0.25540233, ..., 0.26716713, 0.26284607,
        0.28136542]])

In [43]:
from typing import Tuple
def top_k(values: 'np.ndarray', k: int, descending: bool = False) -> Tuple['np.ndarray', 'np.ndarray']:
    """Finds values and indices of the k largest entries for the last dimension.

    :param values: array of distances
    :param k: number of values to retrieve
    :param descending: find top k biggest values
    :return: indices and distances
    """
    if descending:
        values = -values

    if k >= values.shape[1]:
        idx = values.argsort(axis=1)[:, :k]
        values = np.take_along_axis(values, idx, axis=1)
    else:
        idx_ps = values.argpartition(kth=k, axis=1)[:, :k]
        values = np.take_along_axis(values, idx_ps, axis=1)
        idx_fs = values.argsort(axis=1)
        idx = np.take_along_axis(idx_ps, idx_fs, axis=1)
        values = np.take_along_axis(values, idx_fs, axis=1)

    if descending:
        values = -values

    return values, idx
